# VQE
https://dojo.qulacs.org/en/latest/notebooks/5.1_variational_quantum_eigensolver.html
## Overview

In short, this algorithm is used to find the *lowest energy* of a quantum system.
This is useful to solve problems like finding the value of the ground energy of matter, which can be applied to molecules, for example.

It is believed that the properties of molecules and matter are mostly determined by the motion of electrons in them. Therefore, by solving the Schrodinger equation, which is the equation governing electrons.

\begin{equation}
H \ket{\psi} = E \ket{\psi}
\end{equation}

the properties of molecules and matter can be revealed by calculation. Here $H$ is an operator (matrix) called the Hamiltonian, which is determined by the details of the system, such as the shape of the molecules. As can be seen from the form $H|ψ⟩=E|ψ⟩$, solving the Schrodinger equation is equivalent to solving the eigenvalue problem for the Hamiltonian $H$ and finding the eigenvalues $E_{i}$ and the corresponding eigenvectors (also called eigenstates) $|ϕ_{i}⟩$. The eigenvalue $E_{i}$ is the energy of the eigenstate $|ϕ_{i}⟩$.

Unless we consider extreme environments, electrons are usually in the lowest energy state, i.e., the ground state. Therefore, among the eigenstates, the ground state is of particular interest.

The variational method is a powerful method for finding the ground state of a Hamiltonian of very large dimension. The variational method uses the fact that for any given state |ψ⟩, its energy expectation value will always be higher than the ground energy E_{0}, i.e.,

\begin{equation}
\bra{\psi}H\ket{\psi} >= E_{0}
\end{equation}

In fact, if we were to bring states at random, the probability of finding a state close to the ground state would be exponentially small in relation to the size of the system. Therefore, the usual approach is to construct a parameterized quantum state |ψ(θ)⟩ (where θ is a parameter) based on physical and chemical intuition and experience, and find the θ that minimizes

\begin{equation}
\bra{\psi(θ)}H\ket{\psi(θ)} 
\end{equation}


## The algorithm

The algorithm is as follows:

1. Generate a quantum state $\ket{\psi(θ)}$ on a quantum computer.
2. Measure $⟨H(θ)⟩=⟨ψ(θ)|H|ψ(θ)⟩$
3. Based on the measurement results, determine the θ such that ⟨ψ(θ)|H|ψ(θ)⟩ becomes smaller using a classical computer.


This is repeated until ⟨ψ(θ)|H|ψ(θ)⟩ converges to obtain an approximate ground state.




## Example implementation

We'll:
1. Define a simple 2-qubit Hamiltonian (e.g., for H₂ in a toy example).
2. Use a parameterized ansatz.
3. Measure expectation value of energy.
4. Use a classical optimizer to find the ground state energy.


In [7]:
import sys
!{sys.executable} -m pip install qiskit
!{sys.executable} -m pip install qiskit-aer


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\jorgem\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\jorgem\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### 1.Imports

In [28]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
from scipy.optimize import minimize
from qiskit.quantum_info import Statevector



In [37]:
coeffs = [-1.0523732, 0.3979374, -0.3979374, -0.0112801, 0.1809312]
paulis = ["II", "ZI", "IZ", "ZZ", "XX"]
hamiltonian = SparsePauliOp.from_list(list(zip(paulis, coeffs)))

In [38]:
def ansatz(theta):
    qc = QuantumCircuit(2)
    qc.ry(theta[0], 0)
    qc.ry(theta[1], 1)
    qc.cx(0, 1)
    return qc

In [39]:
def energy(theta):
    qc = ansatz(theta)
    state = Statevector.from_instruction(qc)
    return np.real(state.expectation_value(hamiltonian))

In [40]:
theta = np.random.rand(2)
learning_rate = 0.2
steps = 100
delta = 1e-4

In [ ]:
for step in range(steps):
    e = energy(theta)
    print(f"Step {step}: θ = {theta}, Energy = {e:.8f}")
    ansatz(theta).draw('mpl')

    # Finite difference gradient
    grad = np.zeros_like(theta)
    for i in range(len(theta)):
        plus = theta.copy(); plus[i] += delta
        minus = theta.copy(); minus[i] -= delta
        grad[i] = (energy(plus) - energy(minus)) / (2 * delta)
    
    theta -= learning_rate * grad

Step 0: θ = [0.68108015 0.96754275], Energy = -1.07861613
Step 1: θ = [0.63128558 1.01660236], Energy = -1.10371305
Step 2: θ = [0.57982132 1.06931615], Energy = -1.13153456
Step 3: θ = [0.52690685 1.12571982], Energy = -1.16212426
Step 4: θ = [0.47283749 1.1857744 ], Energy = -1.19542806
Step 5: θ = [0.41798932 1.24935137], Energy = -1.23127094
Step 6: θ = [0.36281885 1.31622093], Energy = -1.26934074
Step 7: θ = [0.3078553  1.38604587], Energy = -1.30918428
Step 8: θ = [0.25368428 1.45838324], Energy = -1.35022054
Step 9: θ = [0.20092261 1.53269543], Energy = -1.39177269
Step 10: θ = [0.15018581 1.60837086], Energy = -1.43311665
Step 11: θ = [0.10205155 1.68475248], Energy = -1.47353921
Step 12: θ = [0.05702369 1.76117098], Energy = -1.51239554
Step 13: θ = [0.01550204 1.8369783 ], Energy = -1.54915564
Step 14: θ = [-0.0222381   1.91157709], Energy = -1.58343234
Step 15: θ = [-0.05605418  1.98444298], Energy = -1.61498833
Step 16: θ = [-0.08593246  2.05513793], Energy = -1.64372488
S

In [42]:
print("\nFinal θ:", theta)
print("Final estimated ground state energy:", energy(theta))


Final θ: [-0.2235381   3.13990847]
Final estimated ground state energy: -1.8572744059532635
